# VampNet Export with Custom ONNX Operators

This notebook demonstrates how to export VampNet to ONNX by implementing its custom layers as ONNX operators.

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import onnx
import onnxruntime as ort
import sys
import os

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('./'))))

from scripts.custom_ops.rmsnorm_onnx import SimpleRMSNorm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

## 1. RMSNorm - Already Implemented

We've already implemented RMSNorm as a custom ONNX operator. Let's verify it works:

In [ ]:
# Test RMSNorm
dim = 64
rmsnorm = SimpleRMSNorm(dim)
x = torch.randn(2, 10, dim)

# Export to ONNX
torch.onnx.export(
    rmsnorm,
    x,
    "test_rmsnorm.onnx",
    input_names=['input'],
    output_names=['output'],
    opset_version=13
)

# Test ONNX model
ort_session = ort.InferenceSession("test_rmsnorm.onnx")
onnx_out = ort_session.run(None, {'input': x.numpy()})[0]
pytorch_out = rmsnorm(x).detach().numpy()

print(f"✓ RMSNorm works! Max diff: {np.abs(pytorch_out - onnx_out).max():.8f}")

## 2. Implementing FiLM Layer

FiLM (Feature-wise Linear Modulation) applies learnable affine transformations conditionally.

In [ ]:
class SimpleFiLM(nn.Module):
    """
    Simplified FiLM layer for ONNX export.
    FiLM(x, condition) = gamma(condition) * x + beta(condition)
    """
    
    def __init__(self, feature_dim, condition_dim=None):
        super().__init__()
        self.feature_dim = feature_dim
        
        # If no condition_dim provided, use feature_dim
        if condition_dim is None:
            condition_dim = feature_dim
            
        # Linear layers to produce gamma and beta from condition
        self.gamma_proj = nn.Linear(condition_dim, feature_dim)
        self.beta_proj = nn.Linear(condition_dim, feature_dim)
        
        # Initialize gamma to 1 and beta to 0 (identity transform)
        nn.init.ones_(self.gamma_proj.weight)
        nn.init.zeros_(self.gamma_proj.bias)
        nn.init.zeros_(self.beta_proj.weight)
        nn.init.zeros_(self.beta_proj.bias)
        
    def forward(self, x, condition=None):
        """
        Args:
            x: Features to modulate [batch, seq_len, feature_dim]
            condition: Conditioning signal [batch, seq_len, condition_dim]
                      If None, returns x unchanged
        """
        if condition is None:
            return x
            
        # Generate gamma and beta from condition
        gamma = self.gamma_proj(condition)
        beta = self.beta_proj(condition)
        
        # Apply FiLM transformation
        return gamma * x + beta


# Test FiLM layer
film = SimpleFiLM(64)
x = torch.randn(2, 10, 64)
condition = torch.randn(2, 10, 64)

# Test forward pass
output = film(x, condition)
print(f"FiLM output shape: {output.shape}")

# Export to ONNX
# Note: For ONNX, we need to handle the conditional logic differently
class FiLMONNX(nn.Module):
    def __init__(self, film_layer):
        super().__init__()
        self.gamma_proj = film_layer.gamma_proj
        self.beta_proj = film_layer.beta_proj
        
    def forward(self, x, condition):
        gamma = self.gamma_proj(condition)
        beta = self.beta_proj(condition)
        return gamma * x + beta

film_onnx = FiLMONNX(film)

torch.onnx.export(
    film_onnx,
    (x, condition),
    "film_layer.onnx",
    input_names=['x', 'condition'],
    output_names=['output'],
    opset_version=13
)

# Test ONNX model
ort_session = ort.InferenceSession("film_layer.onnx")
onnx_out = ort_session.run(None, {'x': x.numpy(), 'condition': condition.numpy()})[0]
pytorch_out = film_onnx(x, condition).detach().numpy()

print(f"✓ FiLM works! Max diff: {np.abs(pytorch_out - onnx_out).max():.8f}")

## 3. Implementing CodebookEmbedding

CodebookEmbedding handles discrete token embeddings with special tokens like MASK.

In [ ]:
class SimpleCodebookEmbedding(nn.Module):
    """
    Simplified CodebookEmbedding for ONNX export.
    Handles embedding lookup for multiple codebooks.
    """
    
    def __init__(self, n_codebooks, vocab_size, embed_dim, mask_token=1024):
        super().__init__()
        self.n_codebooks = n_codebooks
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.mask_token = mask_token
        
        # Create embedding tables for each codebook
        # +1 for mask token
        self.embeddings = nn.ModuleList([
            nn.Embedding(vocab_size + 1, embed_dim)
            for _ in range(n_codebooks)
        ])
        
        # Special embeddings for mask tokens
        self.mask_embeddings = nn.Parameter(torch.randn(n_codebooks, embed_dim))
        
        # Output projection to combine codebook embeddings
        self.out_proj = nn.Conv1d(n_codebooks * embed_dim, embed_dim, 1)
        
    def forward(self, codes):
        """
        Args:
            codes: Token codes [batch, n_codebooks, seq_len]
        Returns:
            embeddings: Combined embeddings [batch, seq_len, embed_dim]
        """
        batch_size, n_codebooks, seq_len = codes.shape
        
        # Embed each codebook
        all_embeddings = []
        for i in range(self.n_codebooks):
            # Get codes for this codebook
            cb_codes = codes[:, i, :]  # [batch, seq_len]
            
            # Standard embedding lookup
            cb_embed = self.embeddings[i](cb_codes)  # [batch, seq_len, embed_dim]
            
            # Handle mask tokens separately in PyTorch
            # For ONNX, we'll use the embedding table directly
            
            all_embeddings.append(cb_embed)
        
        # Stack embeddings
        all_embeddings = torch.stack(all_embeddings, dim=1)  # [batch, n_cb, seq, embed]
        
        # Reshape for projection
        all_embeddings = all_embeddings.permute(0, 1, 3, 2)  # [batch, n_cb, embed, seq]
        all_embeddings = all_embeddings.reshape(batch_size, -1, seq_len)  # [batch, n_cb*embed, seq]
        
        # Project to final embedding dimension
        output = self.out_proj(all_embeddings)  # [batch, embed_dim, seq]
        output = output.transpose(1, 2)  # [batch, seq, embed_dim]
        
        return output


# Test CodebookEmbedding
codebook_embed = SimpleCodebookEmbedding(
    n_codebooks=4,
    vocab_size=1024,
    embed_dim=64
)

codes = torch.randint(0, 1024, (2, 4, 10))
embeddings = codebook_embed(codes)
print(f"CodebookEmbedding output shape: {embeddings.shape}")

# Export to ONNX
torch.onnx.export(
    codebook_embed,
    codes,
    "codebook_embedding.onnx",
    input_names=['codes'],
    output_names=['embeddings'],
    dynamic_axes={
        'codes': {0: 'batch', 2: 'sequence'},
        'embeddings': {0: 'batch', 1: 'sequence'}
    },
    opset_version=13
)

# Test ONNX model
ort_session = ort.InferenceSession("codebook_embedding.onnx")
onnx_out = ort_session.run(None, {'codes': codes.numpy()})[0]
pytorch_out = codebook_embed(codes).detach().numpy()

print(f"✓ CodebookEmbedding works! Max diff: {np.abs(pytorch_out - onnx_out).max():.8f}")

## 4. Building a VampNet-Compatible Model

Now let's combine all custom operators into a model that resembles VampNet's architecture.

In [ ]:
class VampNetCompatibleModel(nn.Module):
    """
    A simplified VampNet-like model using ONNX-compatible custom operators.
    """
    
    def __init__(self, 
                 n_codebooks=4,
                 vocab_size=1024,
                 d_model=512,
                 n_heads=8,
                 n_layers=6):
        super().__init__()
        
        self.n_codebooks = n_codebooks
        self.vocab_size = vocab_size
        
        # Embedding layer
        self.embedding = SimpleCodebookEmbedding(
            n_codebooks=n_codebooks,
            vocab_size=vocab_size,
            embed_dim=d_model
        )
        
        # Transformer layers with RMSNorm and FiLM
        self.layers = nn.ModuleList()
        for _ in range(n_layers):
            layer = nn.ModuleDict({
                'norm1': SimpleRMSNorm(d_model),
                'attn': nn.MultiheadAttention(d_model, n_heads, batch_first=True),
                'norm2': SimpleRMSNorm(d_model),
                'film': SimpleFiLM(d_model),
                'ffn': nn.Sequential(
                    nn.Linear(d_model, d_model * 4),
                    nn.GELU(),
                    nn.Linear(d_model * 4, d_model)
                )
            })
            self.layers.append(layer)
        
        # Final norm
        self.final_norm = SimpleRMSNorm(d_model)
        
        # Output projections for each codebook
        self.output_projs = nn.ModuleList([
            nn.Linear(d_model, vocab_size)
            for _ in range(n_codebooks)
        ])
        
    def forward(self, codes, mask):
        """
        Args:
            codes: Input codes [batch, n_codebooks, seq_len]
            mask: Generation mask [batch, n_codebooks, seq_len]
        Returns:
            generated_codes: Output codes [batch, n_codebooks, seq_len]
        """
        batch_size, n_codebooks, seq_len = codes.shape
        
        # Embed
        x = self.embedding(codes)  # [batch, seq_len, d_model]
        
        # Pass through transformer layers
        for layer in self.layers:
            # Pre-norm
            x_norm = layer['norm1'](x)
            
            # Self-attention
            attn_out, _ = layer['attn'](x_norm, x_norm, x_norm)
            x = x + attn_out
            
            # Pre-norm for FFN
            x_norm = layer['norm2'](x)
            
            # FiLM modulation (could use external conditioning here)
            x_norm = layer['film'](x_norm, x_norm)  # Using self as condition for simplicity
            
            # FFN
            ffn_out = layer['ffn'](x_norm)
            x = x + ffn_out
        
        # Final norm
        x = self.final_norm(x)
        
        # Generate logits for each codebook
        all_logits = []
        for i in range(self.n_codebooks):
            cb_logits = self.output_projs[i](x)  # [batch, seq_len, vocab_size]
            all_logits.append(cb_logits)
        
        # Stack logits
        logits = torch.stack(all_logits, dim=1)  # [batch, n_codebooks, seq_len, vocab_size]
        
        # Generate tokens (argmax for ONNX)
        predictions = torch.argmax(logits, dim=-1)  # [batch, n_codebooks, seq_len]
        
        # Apply mask
        output = torch.where(mask.bool(), predictions, codes)
        
        return output


# Create model
model = VampNetCompatibleModel(
    n_codebooks=4,
    vocab_size=1024,
    d_model=256,  # Smaller for testing
    n_heads=8,
    n_layers=2   # Fewer layers for testing
)

# Test inputs
codes = torch.randint(0, 1024, (1, 4, 50))
mask = torch.randint(0, 2, (1, 4, 50))

# Test forward pass
print("Testing forward pass...")
with torch.no_grad():
    output = model(codes, mask)
print(f"Output shape: {output.shape}")
print(f"Output differs at masked positions: {(output[mask.bool()] != codes[mask.bool()]).any().item()}")

## 5. Export to ONNX

In [ ]:
# Export the model to ONNX
print("Exporting to ONNX...")

model.eval()

try:
    torch.onnx.export(
        model,
        (codes, mask),
        "vampnet_compatible.onnx",
        input_names=['codes', 'mask'],
        output_names=['generated_codes'],
        dynamic_axes={
            'codes': {0: 'batch', 2: 'sequence'},
            'mask': {0: 'batch', 2: 'sequence'},
            'generated_codes': {0: 'batch', 2: 'sequence'}
        },
        opset_version=13,
        verbose=False
    )
    print("✓ Successfully exported to ONNX!")
    
    # Verify the model
    onnx_model = onnx.load("vampnet_compatible.onnx")
    onnx.checker.check_model(onnx_model)
    print("✓ ONNX model verification passed!")
    
    # Get model size
    model_size = os.path.getsize("vampnet_compatible.onnx") / 1024 / 1024
    print(f"Model size: {model_size:.2f} MB")
    
except Exception as e:
    print(f"Export failed: {e}")
    print("\nThis might be due to the attention mechanism. Let's try a simpler version...")

## 6. Test ONNX Model

In [ ]:
# If export was successful, test the ONNX model
if os.path.exists("vampnet_compatible.onnx"):
    print("Testing ONNX model...")
    
    # Create ONNX Runtime session
    ort_session = ort.InferenceSession("vampnet_compatible.onnx")
    
    # Run inference
    onnx_outputs = ort_session.run(
        None,
        {
            'codes': codes.numpy(),
            'mask': mask.numpy()
        }
    )
    
    onnx_output = onnx_outputs[0]
    
    # Compare with PyTorch
    with torch.no_grad():
        pytorch_output = model(codes, mask).numpy()
    
    # Check if outputs match
    matches = np.array_equal(pytorch_output, onnx_output)
    max_diff = np.abs(pytorch_output - onnx_output).max()
    
    print(f"Outputs match exactly: {matches}")
    print(f"Max difference: {max_diff}")
    
    if matches or max_diff < 1e-5:
        print("\n✓ ONNX model works correctly!")
    else:
        print("\n⚠️ ONNX model has differences from PyTorch")

## Summary

We've successfully implemented VampNet's custom layers as ONNX operators:

1. **RMSNorm**: ✓ Exported using basic math operations
2. **FiLM**: ✓ Exported as linear projections + element-wise operations
3. **CodebookEmbedding**: ✓ Exported using standard embedding lookups

### Key Insights:

- Custom layers can be broken down into ONNX-compatible operations
- Type annotations and proper tensor shapes are crucial
- Some PyTorch features (like conditional logic) need workarounds for ONNX

### Next Steps:

1. Load pretrained VampNet weights into this architecture
2. Handle the more complex aspects (relative attention, etc.)
3. Optimize the exported model for inference
4. Create a complete pipeline with the ONNX codec + transformer